# Filtrage temporel dans la gamme synoptique et énergie cinétique des perturbations (EKE)

**Auteur : FERRY Frédéric (DESR/ENM/C3M) - septembre 2022**

Les fichiers de données au format netcdf (données quotidiennes NCEP du vent zonal et du vent méridien à 300hPa pour l'année 2021) doivent être téléchargés et placés dans le répertoire data :

ftp://ftp2.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/pressure/

In [ ]:
import os
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import matplotlib.path as mpath

import pandas as pd

from tqdm import tqdm

In [ ]:
dir_data='./daily_ncep/'
dir_figs='./figs/'
dir_anim='./anim/'
if not os.path.exists(dir_figs):
    os.makedirs(dir_figs)
if not os.path.exists(dir_anim):
    os.makedirs(dir_anim)

# Traitement des données u, v, z

In [ ]:
year1='1980'
year2='2021'

fu    = xr.open_dataset(dir_data+'uwnd300.1979-2021.nc').sel(time=slice(year1,year2))
fv    = xr.open_dataset(dir_data+'vwnd300.1979-2021.nc').sel(time=slice(year1,year2))
fz    = xr.open_dataset(dir_data+'hgt500.1979-2021.nc').sel(time=slice(year1,year2))
print(fu)
lat  = fu.lat.values
lon  = fu.lon.values

In [ ]:
print(' ----- Computing climatology ----- ')
fu_clim = fu.groupby('time.dayofyear').mean('time')
fv_clim = fv.groupby('time.dayofyear').mean('time')
fz_clim = fz.groupby('time.dayofyear').mean('time')

print(fu_clim)

print(' ----- Computing daily anomalies ----- ')
fuprime = fu.groupby('time.dayofyear') - fu_clim
fvprime = fv.groupby('time.dayofyear') - fv_clim
fzprime = fz.groupby('time.dayofyear') - fz_clim

print(fuprime)

# Filtrage de la gamme synoptique 2-10j

In [ ]:
u=fuprime['uwnd']
v=fvprime['vwnd']
z=fzprime['hgt']
print(u.shape)

In [ ]:
def low_pass_weights(window, cutoff):
    """Calculate weights for a low pass Lanczos filter.
    Args:
    window: int
        The length of the filter window.
    cutoff: float
        The cutoff frequency in inverse time steps.
    """
    order = ((window - 1) // 2 ) + 1
    nwts = 2 * order + 1
    w = np.zeros([nwts])
    n = nwts // 2
    w[n] = 2 * cutoff
    k = np.arange(1., n)
    sigma = np.sin(np.pi * k / n) * n / (np.pi * k)
    firstfactor = np.sin(2. * np.pi * cutoff * k) / (np.pi * k)
    w[n-1:0:-1] = firstfactor * sigma
    w[n+1:-1] = firstfactor * sigma
    return w[1:-1]

In [ ]:
# window length for filters
window = 50

day1=2.
day2=8.

# construct low pass filters
hfw = low_pass_weights(window, 1. / day1)
lfw = low_pass_weights(window, 1. / day2)
weight_high = xr.DataArray(hfw, dims = ['window'])
weight_low = xr.DataArray(lfw, dims = ['window'])

In [ ]:
# apply the filters using the rolling method with the weights
lowpass_hf = u.rolling(time = len(hfw), center = True).construct('window').dot(weight_high)
lowpass_lf = u.rolling(time = len(lfw), center = True).construct('window').dot(weight_low)
# the bandpass is the difference of two lowpass filters.
u2_10= lowpass_hf - lowpass_lf

In [ ]:
# apply the filters using the rolling method with the weights
lowpass_hf = v.rolling(time = len(hfw), center = True).construct('window').dot(weight_high)
lowpass_lf = v.rolling(time = len(lfw), center = True).construct('window').dot(weight_low)
# the bandpass is the difference of two lowpass filters.
v2_10= lowpass_hf - lowpass_lf

In [ ]:
# apply the filters using the rolling method with the weights
lowpass_hf = z.rolling(time = len(hfw), center = True).construct('window').dot(weight_high)
lowpass_lf = z.rolling(time = len(lfw), center = True).construct('window').dot(weight_low)
# the bandpass is the difference of two lowpass filters.
z2_10= lowpass_hf - lowpass_lf

In [ ]:
print(z2_10.shape)

# Illustration d'une séquence particulière (anomalie VS anomalie filtrée)

In [ ]:
date1='2021-01-01'
date2='2021-01-31'

fz_days=fz.sel(time=slice(date1,date2))
dataz1 = fz_days['hgt']

fz_days_anom=fzprime.sel(time=slice(date1,date2))
dataz2 = fz_days_anom['hgt']

dataz3=z2_10.sel(time=slice(date1,date2))

time  = fz_days.time.values

In [ ]:
time_str=[x for x in range(len(time))]
date_str=[x for x in range(len(time))]

for i in range(len(time)):
	time_str[i] = str(time[i])
	date_str[i] = time_str[i][0:10]

print(date_str)

In [ ]:
def lonflip(da):
    lon_name = 'lon'
    da['_longitude_adjusted'] = xr.where(
        da[lon_name] > 180,
        da[lon_name] - 360,
        da[lon_name])
    da = (
        da
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(da._longitude_adjusted)})
        .drop(lon_name))
    da = da.rename({'_longitude_adjusted': lon_name})
    return da

In [ ]:
dataz1=lonflip(dataz1)
dataz2=lonflip(dataz2)
dataz3=lonflip(dataz3)

lat1  = dataz1.lat.values
lon1  = dataz1.lon.values

In [ ]:
print("Original shape -", dataz1.shape)
lon0 = dataz1.coords['lon']
lon_idx = dataz1.dims.index('lon')
dataz1, lon2 = add_cyclic_point(dataz1.values, coord=lon0, axis=lon_idx)
dataz2, lon2 = add_cyclic_point(dataz2.values, coord=lon0, axis=lon_idx)
dataz3, lon2 = add_cyclic_point(dataz3.values, coord=lon0, axis=lon_idx)
print("New shape -", dataz1.shape)

In [ ]:
levels_z1 = np.arange(4800,6050,50)
levels_z2 = np.arange(-500,550,50)
cmap_z1='jet'
cmap_z2='RdBu_r'

In [ ]:
def make_animation(gif_filepath):
    from PIL import Image
    import os
    from IPython.display import Image as IPImage
    from IPython.display import display
    import time
    
    image_folder = './anim/' # répertoire contenant les fichiers PNG
    output_file = gif_filepath # nom du fichier de sortie
    animation_speed = 0.9 # vitesse de l'animation en secondes
    
    # Liste tous les fichiers PNG dans le répertoire image_folder
    files = sorted(os.listdir(image_folder))
    image_files = [f for f in files if f.endswith('.png')]
    
    # Ouvre chaque fichier PNG et ajoute l'image à une liste
    images = []
    for filename in image_files:
        img = Image.open(os.path.join(image_folder, filename))
        images.append(img)
    
    # Crée l'animation GIF
    images[0].save(output_file, save_all=True, append_images=images[1:], duration=int(animation_speed*1000), loop=0)
    # Affiche l'animation GIF dans Jupyter
    with open(output_file,'rb') as f:
        display(IPImage(data=f.read(), format='png'))
    # Efface les fichiers PNG
    for filename in image_files:
        os.remove(image_folder+filename)

In [ ]:
proj=ccrs.NorthPolarStereo()
bounds = [(-180, 180, 30, 90)]

plt_title1 = 'Geopotential height (mgp) at 500 hPa : '

for i in tqdm(range(len(time))):
    #print(date_str[i])
    fig = plt.figure(figsize=(15., 8.))
    fig.suptitle(plt_title1+date_str[i], fontsize=16)
    ax = fig.add_subplot(1, 2, 1, projection=proj)
    ax.set_title('Daily field', loc='center')
    ax.coastlines()
    
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    angle = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(angle), np.cos(angle)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)
    
    cf = ax.contourf(lon2, lat1, dataz1[i,:,:], levels_z1, transform=ccrs.PlateCarree(), cmap=cmap_z1, extend='both')
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.10, extendrect='True')
    cb.set_label('mgp', size='large')
    
    ax = fig.add_subplot(1, 2, 2, projection=proj)
    ax.set_title('Anomaly', loc='center')
    ax.coastlines()
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    angle = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(angle), np.cos(angle)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    cf = ax.contourf(lon2, lat1, dataz2[i,:,:], levels_z2, transform=ccrs.PlateCarree(), cmap=cmap_z2, extend='both')
    
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.10, extendrect='True')
    cb.set_label('mgp', size='large')
    
    plt.clf()
    plt.close()
    
    figname=dir_anim+'Z500_polar_'+date_str[i]
    fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
gif_filepath = dir_anim+'Z500_polar_'+date1+'-'+date2+'.gif'
make_animation(gif_filepath)

In [ ]:
proj=ccrs.NorthPolarStereo()
bounds = [(-180, 180, 30, 90)]

plt_title1 = 'Geopotential height (mgp) at 500 hPa : '

for i in tqdm(range(len(time))):
    #print(date_str[i])
    fig = plt.figure(figsize=(15., 8.))
    fig.suptitle(plt_title1+date_str[i], fontsize=16)
    ax = fig.add_subplot(1, 2, 1, projection=proj)
    ax.set_title('Daily field', loc='center')
    ax.coastlines()
    
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    angle = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(angle), np.cos(angle)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)
    
    cf = ax.contourf(lon2, lat1, dataz1[i,:,:], levels_z1, transform=ccrs.PlateCarree(), cmap=cmap_z1, extend='both')
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.10, extendrect='True')
    cb.set_label('mgp', size='large')
    
    ax = fig.add_subplot(1, 2, 2, projection=proj)
    ax.set_title('2-8 day filtered anomaly', loc='center')
    ax.coastlines()
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    angle = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(angle), np.cos(angle)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    cf = ax.contourf(lon2, lat1, dataz3[i,:,:], levels_z2, transform=ccrs.PlateCarree(), cmap=cmap_z2, extend='both')
    
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.10, extendrect='True')
    cb.set_label('mgp', size='large')
    
    plt.close()
    
    figname=dir_anim+'Z500_polar_filtered_'+date_str[i]
    fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
gif_filepath = dir_anim+'Z500_polar_filtered_'+date1+'-'+date2+'.gif'
make_animation(gif_filepath)

# Calcul de l'EKE et climatologies saisonnières

In [ ]:
season_name=input("DJF or JJA ? ")

In [ ]:
# Période totale des données
d1=fu.time.values[0]
d2=fu.time.values[-1]
dates = pd.date_range(d1, d2, freq='D')
print(dates)

# Select Season
if season_name == 'DJF':
    months=np.any([dates.month==12,dates.month==1,dates.month==2],axis=0)
if season_name == 'JJA':
    months=np.any([dates.month==6,dates.month==7,dates.month==8],axis=0)

dates2=dates[months]
print(dates2)

In [ ]:
uprime_season = u2_10.sel(time=dates2)
vprime_season = v2_10.sel(time=dates2)
print(uprime_season)

## Calcul de l'EKE sur les saisons et de la moyenne saisonnière.

<div class="alert alert-danger">
<p>Calculer l'énergie cinétique des perturbations (eke) à partir des données saisonnières filtrées dans la gamme 2-10j (uprime_season et vprime_season). </p>
<p>Calculer la moyenne saisonnière de l'énergie cinétique des pertubations (eke_mean). Attention à ne pas prendre en compte les valeurs manquantes liées au filtrage temporel (skipna = True). </p>
</div>

In [ ]:
eke=
eke_mean=

In [ ]:
if season_name == 'DJF':
    levels=np.arange(40,155,5)
    
if season_name == 'JJA':
    levels=np.arange(40,155,5)
    #levels=np.arange(40,125,5)
    
proj=ccrs.PlateCarree(central_longitude=180)

def plot_background(ax):
    ax.coastlines()
    ax.gridlines(draw_labels=True)
    ax.set_global()
    return ax

fig = plt.figure(figsize=(15., 12.))

ax = fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title('Eddy (2-8 day filtered) Kinetic Energy (EKE) at 300 hPa : '+season_name, fontsize=10)
cf = ax.contourf(lon, lat, eke_mean, levels, transform=ccrs.PlateCarree(), cmap='YlOrRd', extend='max')
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
cb.set_label('$m^2s^{-2}$', size='small')

figname='./figs/eke_'+season_name
fig.savefig(figname+'.png')
plt.show()